## Newsletter Email agents with Handoffs
##### Part 1 (Tools)
- Create 3 agents which will write the emails.
- Convert them into tools also create another tools which will send the push notification


##### Part 2 (Handoff)
- Create 2 agents
    1. Generate a catchy subject line
    2. Convert the email into an HTML.
- Create a Handoff agent  
    This agent will be the email formatter and sender agent.  
    This agent will use the subject_tool, html_tool and send_push_notification as tools
    **NOTE: This agent will have an handoff instruction**
        This instrution will specify what the agent is going to do

##### Part 3 (Sales Manager)
- Create a new agent which will be the Sales Manager
    - Responsible to use all the tools and Handoffs
    - tools -> agent_tools from part 1
    - handoffs -> Part2 handoff agent (will use the subject_tool and html_tool internally)

##### Part 3.1 (Guardrails -> Udpated Sales Manager)  
Guardrails are something we can add at the input (First Agents input) level or at the output (Last agents output) level.  
We can use Pydantic models to check the format of the input.
- Create a Pydantic object 
- Create a guardrail agent which will output the pydantic model object.
    - output_type: Pydantic Model (Important change)
- We can then create a guardrail similar to a tool using a decorator.
    - This guardrail will call the Agent with the input message or any message and check its formatting.
    - The returned object will tell us if the formatting is correct or incorrect.  
    - The output of this object will determine if we trigger the guardrail exception.  
        - openAI SDK allows to return a **GuardrailFunctionOutput** which takes care of some things for us.  
            1. If triggered it returns and exception(tripwire).  
            2. Stops the execution of the agent.  
    Note -> There is a specific formatting of how we trigger this gaurdrail in openAI SDK, check the code for the proper formatting.
- This gaurdrail runner function we will add it to the final prompt we will call it careful_sales_manager

All this will work cohisively to create a well formatted email and send it to the customer.

In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool, input_guardrail, GuardrailFunctionOutput
import os
import requests
from pydantic import BaseModel

In [ ]:
load_dotenv(override=True)

### Part 1 (Tools)

In [ ]:
# PART 1 tools
intruction_1 = """You are a sales agent working for FinAI, a company that provides AI solutions for financial institutions.
You write professional, serious cold emails.
"""
intruction_2 = """You are a humorous, engaging sales agent working for FinAI, a company that provides AI solutions for financial institutions.
You write witty, engaging cols emails that are likely to get a response
"""
intruction_3 = """You are a busy sales agent working for FinAI, a company that provides AI solutions for financial institutions.
You write professional, serious cold emails.
"""
sales_agent_1 = Agent(instructions=intruction_1, name="Professional Sales Agent", model="gpt-4o-mini")
sales_agent_2 = Agent(instructions=intruction_2, name="Humorous Sales Agent", model="gpt-4o-mini")
sales_agent_3 = Agent(instructions=intruction_3, name="Busy Sales Agent", model="gpt-4o-mini")
description = "Write a cold email addressed to 'Dear Bank Manager'"
tool1 = sales_agent_1.as_tool(tool_name="sales_agent_1", tool_description=description)
tool2 = sales_agent_2.as_tool(tool_name="sales_agent_2", tool_description=description)
tool3 = sales_agent_3.as_tool(tool_name="sales_agent_3", tool_description=description)

In [ ]:
# Push notification helper function
def push(text):
    requests.post(
        "https://api.pushover.net/1/messages.json",
        data={
            "token": os.getenv("PUSHOVER_TOKEN"),
            "user": os.getenv("PUSHOVER_USER"),
            "message": text,
        }
    )

In [ ]:
# Tool which send the push notification
@function_tool
def send_push_notification(message: str) -> None:
    """Send a push notification with the given message."""
    push(message)
    return {"status": "success"}

### Part 2 (Handoffs)

In [ ]:
# Part 2 (Handoff)

subject_instruction = "Generate a concise, engaging email subject line for a cold sales email. \
    You are given a message and you are needed to generate a subject line for it."
    
html_instruction = "Generate a professional HTML email body for a cold sales email. \
    You are given a text email body which might have some markdown \
    and you need to convert it to an HTML email body with simple, clear, compelling layout and design."
    
subject_write = Agent(
    instructions=subject_instruction, name="Subject Writer", model="gpt-4o-mini"
)

html_converter = Agent(
    instructions=html_instruction, name="HTML Converter", model="gpt-4o-mini"
)

subject_writer_tool = subject_write.as_tool(tool_name="subject_writer", tool_description=subject_instruction)
html_converter_tool = html_converter.as_tool(tool_name="html_converter", tool_description=html_instruction)



In [ ]:
# Handoff push notification agent
push_notification_instruction = """You are an email formatter and sender. You receive the bosy of an email to be sent.
You first user the subject_writer_tool to write a subject for the email, then use the html_converter_tool to convert the body to HTML.
Finally, you use the send_push_notification tool to send a push notification with the subject and HTML body as text.
"""

handoff_push_agent = Agent(
    name="Handoff Push Notification Agent",
    instructions=push_notification_instruction,
    tools=[subject_writer_tool, html_converter_tool, send_push_notification],
    model="gpt-4o-mini",
    handoff_description="Conver the email to HTML and send it as a push notification"
)


### Part 3 & 3.1 (Upgraded Sales Manager)

In [ ]:
# Part 3.1 
# Guardrail model
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent = Agent(
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)


In [ ]:
# Guardrail as a tool
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output}, tripwire_triggered=is_name_in_message)

In [ ]:
# PART 3

sales_manager_instructions = """
You are a sales manager at FinAI. Your goal is to find the single best cold email using sales_agent tools.

Follow this steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different cold email drafts. Do not proceed until all three drafts are ready.

2. Evaluate and Select: Review the drafts and choose the single best email using your judgement of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.

3. Handoff for Sending: Pass ONLY the winning email draft to the handoff_push_agent(Handoff Push Notification Agent).
The handoff_push_agent will take care of the formatting and sending.

Crucial Rules:
- You must use the sales agent tools to generate the drafts - do not write them yourself.
- You must hand off exactly ONE email to the handoff_push_agent(Handoff Push Notification Agent) - never more than one.
"""

careful_sales_manager = Agent(
    name = "Sales Manager",
    instructions = sales_manager_instructions,
    tools=[tool1, tool2, tool3],
    model="gpt-4o-mini",
    handoffs=[handoff_push_agent],
    input_guardrails=[guardrail_against_name]
)

# This message will raise and expection since the name "Ashutosh" is included in the message.
# message = "Send a cold sales email address to Dead Bank Manager from Ashutosh"

# with trace("Sales Manager"):
#     result = await Runner.run(careful_sales_manager, message)


In [19]:
message = "Sent out a cold sales email addressed to Dear Bank Manager from Head of Business Developement"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)